In [ ]:
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as T
import tifffile

def calculate_ndvi(image):
    """
    Calculate the NDVI value, assuming that the first channel of the image is NIR and the second channel is the red band.
    """
    nir = image[:, :, 80].astype(np.float32)
    red = image[:, :, 38].astype(np.float32)
    ndvi = (nir - red) / (nir + red + 1e-8)  
    return ndvi

def create_SAM_mask_with_ndvi(image_dir, save_dir):
    # Initializing the SAM model
    sam = sam_model_registry["default"](checkpoint="/home/sirui/Diffusion/sam_vit_h_4b8939.pth")
    if torch.cuda.is_available():
        sam = sam.cuda()
    mask_generator = SamAutomaticMaskGenerator(sam, points_per_batch=10)

    resize_transform = T.Resize((256, 256), interpolation=T.InterpolationMode.BILINEAR)

    os.makedirs(save_dir, exist_ok=True)

    for i, img_id in enumerate(os.listdir(image_dir)):
        image_path = os.path.join(image_dir, img_id)
        image = tifffile.imread(image_path)
        print(f"Original shape of {img_id}: {image.shape}")
        image=image.transpose(1, 2, 0)
        sam_image = image[:, :, [38, 23, 5]]

        sam_image = torch.tensor(sam_image, dtype=torch.float32).permute(2, 0, 1) 
        sam_image = resize_transform(sam_image) 
        sam_image = sam_image.permute(1, 2, 0).numpy() 

        sam_image_min, sam_image_max = sam_image.min(), sam_image.max()
        sam_image = ((sam_image - sam_image_min) / (sam_image_max - sam_image_min)) * 255
        sam_image = sam_image.astype(np.uint8)

        masks = mask_generator.generate(sam_image)

        index_map = np.zeros(sam_image.shape[:2], dtype=np.int32)
        for j, mask in enumerate(masks):
            seg = np.array(mask['segmentation'])
            index_map[seg] = j + 1
        image_tensor = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)  
        image_tensor = resize_transform(image_tensor) 
        image_resized = image_tensor.permute(1, 2, 0).numpy()  

        ndvi = calculate_ndvi(image_resized)
        print(ndvi.shape)

        classification_mask = np.zeros_like(ndvi)


        classification_mask[ndvi > 0.6] = 0    #vegatation
        classification_mask[(ndvi <= 0.6) & (ndvi > 0)] = 0.5 #aritficial surface
        classification_mask[ndvi <= 0.2] = 1   #building

        final_mask = classification_mask.copy()
        for j, mask in enumerate(masks):
            seg = np.array(mask['segmentation'])
            final_mask[seg] = classification_mask[seg].mean()

        save_path = os.path.join(save_dir, img_id.split('.')[0] + ".npy")
        np.save(save_path, final_mask)

        

        print(f"Processed {img_id}: Final mask shape {final_mask.shape}")
        
create_SAM_mask_with_ndvi("/home/sirui/Diffusion/hyspex/test/lq", "/home/sirui/Diffusion/hyspex/test/mask")#process the low resolution input only


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as T
import tifffile
import cv2
from skimage.util import random_noise
from skimage import feature


def apply_edge_detection(image, low_threshold=50, high_threshold=150):
    """
    Apply Canny edge detection, setting edge values ​​to 1 and other values ​​to 0.
    """
    gray_image = cv2.cvtColor(image[:,:,[40,30,20]], cv2.COLOR_BGR2GRAY) 
    edges = cv2.Canny(gray_image, low_threshold, high_threshold)  
    edge_mask = (edges > 0).astype(np.float32)  
    return edge_mask
def apply_gaussian_blur(image, kernel_size=5):
    blurred_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
    return blurred_image
    
def create_edge_mask_with_ndvi(image_dir, save_dir):
    resize_transform = T.Resize((256, 256), interpolation=T.InterpolationMode.BILINEAR)
    os.makedirs(save_dir, exist_ok=True)

    for i, img_id in enumerate(os.listdir(image_dir)):
        image_path = os.path.join(image_dir, img_id)
        image = tifffile.imread(image_path)
        print(f"Original shape of {img_id}: {image.shape}")
        image = image.transpose(1, 2, 0) 

        image_min, image_max = image.min(), image.max()
        image = ((image - image_min) / (image_max - image_min)) * 255
        image = image.astype(np.uint8)

        image_tensor = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)  
        image_tensor = resize_transform(image_tensor) 
        resized_image = image_tensor.permute(1, 2, 0).numpy().astype(np.uint8)  
        gray_image = cv2.cvtColor(resized_image[:,:,[40,30,20]], cv2.COLOR_BGR2GRAY)  

        edge_mask = apply_edge_detection(resized_image)
        print(f"Edge mask shape: {edge_mask.shape}")
        input = random_noise(gray_image, mode='speckle', mean=0.07)

# Compute the Canny filter for two values of sigma
        edges1 = feature.canny(input)
        edges2 = feature.canny(input, sigma=2)
        final_mask = edges1.copy()
        final_mask[edges1 > 0] = 1 

        save_path = os.path.join(save_dir, img_id.split('.')[0] + ".npy")
        np.save(save_path, final_mask)

        print(f"Processed {img_id}: Final mask shape {final_mask.shape}")
        
create_edge_mask_with_ndvi("/home/sirui/Diffusion/hyspex/train/lq", "/home/sirui/Diffusion/hyspex/train/edge")#process the low resolution input only
